In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from transformers import RobertaTokenizer, RobertaModel, ViTFeatureExtractor, ViTModel
from PIL import Image
import joblib
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.nn import ReLU, Sigmoid, LeakyReLU
from transformers import AutoTokenizer, AutoModel, ViTFeatureExtractor, ViTModel
# 定义设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 读取CSV文件
df = pd.read_csv("最终的记录.csv", encoding='utf-8', encoding_errors='ignore')

# 重命名列（确保与之前一致）
df.columns = [
    '商品名称', '一级种类', '二级种类', '图片地址', 'sku', '文本描述', '折扣率',
    '折扣价', '价格', '星级', '销量', '收入', '评论', 'cc-1', 'cc-2',
    'cc-3', 'DRC-1', 'DRC-2', 'DRC-3', 'RCV-1', 'RCV-2',
    'RCV-3', 'RSV-1', 'RSV-2', 'RSV-3'
]
# 重置索引
df = df.reset_index(drop=True)

# 转换数据类型
numeric_columns = ['折扣率', '折扣价', '价格', '销量', '收入']
for col in numeric_columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# 重新计算收入（如果需要）
df['收入'] = df['折扣价'] * df['销量']
df['收入'] = np.log10(df['收入'].replace(0, 1))  # 避免 log10(0) 错误
df['折扣率'] = 1 - df['折扣价'] / df['价格']

# 数值特征
numerical_features = ['折扣价', '折扣率', '价格']

# 标准化数值特征
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# 保存 Scaler 以备后续使用
joblib.dump(scaler, 'scaler.pkl')

import torchvision.models as models
import torchvision
from torchvision import transforms

# 图像特征提取
def vit_encoding(image_dir, num_images=2194, device='cpu'):
    vit_list = []
    model_name = "/root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3"
    feature_extractor = ViTFeatureExtractor.from_pretrained(model_name)
    model = ViTModel.from_pretrained(model_name).to(device).eval()

    def process_image(image_path):
        try:
            image = Image.open(image_path).convert('RGB')
            return feature_extractor(images=image, return_tensors="pt")
        except Exception as e:
            print(f"无法打开图片 {image_path}: {e}")
            # 返回全零张量以保持尺寸一致
            return feature_extractor(images=Image.new('RGB', (224, 224), (0, 0, 0)), return_tensors="pt")

    for i in tqdm(range(num_images), desc="提取图像特征"):
        image_path = os.path.join(image_dir, f"{i}.jpg")
        inputs = process_image(image_path)
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = model(**inputs)
            cls_token = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # [1, hidden_size]
            vit_list.append(cls_token.squeeze())
    
    vit_np = np.array(vit_list)  # [num_images, hidden_size]
    
    # PCA降维
    vit_pca = PCA(n_components=50)
    vit_features_pca = vit_pca.fit_transform(vit_np)
    
    # 保存PCA模型
    joblib.dump(vit_pca, 'vit_pca.pkl')
    
    return vit_features_pca

# 使用您的图像路径定义
image_directory = r"/root/img"
vit_features = vit_encoding(image_directory, num_images=len(df))
df['vit_features'] = list(vit_features)


# 初始化 BERT 模型和分词器
bert_model_name = r"/root/.cache/huggingface/hub/models--google-bert--bert-base-uncased/snapshots/86b5e0934494bd15c9632b12f734a8a67f723594"
tokenizer_bert = AutoTokenizer.from_pretrained(bert_model_name)
model_bert = AutoModel.from_pretrained(bert_model_name, from_tf=False)  # 根据实际情况设置 from_tf
model_bert.to(device)
model_bert.eval()  # 设置为评估模式

def extract_text_features(texts, model, tokenizer, device, batch_size=32, max_length=128):
    all_features = []
    num_samples = len(texts)
    num_batches = int(np.ceil(num_samples / batch_size))
    
    with torch.no_grad():  # 禁用梯度计算
        for batch_idx in tqdm(range(num_batches), desc="提取 BERT 文本特征"):
            start_idx = batch_idx * batch_size
            end_idx = min(start_idx + batch_size, num_samples)
            batch_texts = texts[start_idx:end_idx]
            
            # Tokenize with fixed max_length and padding to max_length
            inputs = tokenizer(
                batch_texts,
                return_tensors='pt',
                padding='max_length',  # 固定填充到 max_length
                truncation=True,
                max_length=max_length,
            )
            inputs = {key: val.to(device) for key, val in inputs.items()}
            
            # 获取模型输出
            outputs = model(**inputs)
            
            # 对最后一个隐藏状态进行平均池化，得到每个样本的固定维度特征
            encoded_text = outputs.last_hidden_state.mean(dim=1).cpu().numpy()  # [batch_size, hidden_size]
            
            # 添加到特征列表
            all_features.append(encoded_text)
    
    # 将所有批次的特征堆叠起来，得到 [num_samples, hidden_size]
    all_features = np.vstack(all_features)
    return all_features

# 示例 DataFrame（请根据您的数据来源调整）
# df = pd.read_csv('your_data.csv')  # 根据您的数据来源加载 DataFrame

# 确保所有文本字段为字符串类型
text_columns = ['商品名称', '文本描述', '一级种类', '二级种类', 'cc-2']
for col in text_columns:
    df[col] = df[col].astype(str)

# 构建“其他文本”特征
other_texts = (
    df['商品名称'] + " " +
    df['一级种类'] + " " +
    df['二级种类'] + " " +
    df['cc-1']
).tolist()

# 构建“文本描述”特征
text_descs = df['文本描述'].tolist()

# 提取 BERT 的“其他文本”特征
bert_other_text_features = extract_text_features(
    other_texts,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['other_text_features'] = list(bert_other_text_features)

# 提取 BERT 的“文本描述”特征
bert_text_desc_features = extract_text_features(
    text_descs,
    model_bert,
    tokenizer_bert,
    device,
    batch_size=32,
    max_length=128
)
df['text_desc_features'] = list(bert_text_desc_features)

print("BERT 文本特征提取完成。")

# 合并所有特征
all_features = np.concatenate([
    df[numerical_features].values,
    np.stack(df['text_desc_features'].values),
    np.stack(df['vit_features'].values),
    np.stack(df['other_text_features'].values)
], axis=1)

# 定义目标变量
targets = df['收入'].to_numpy()

/root/miniconda3/lib/python3.12/site-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Some weights of ViTModel were not initialized from the model checkpoint at /root/.cache/huggingface/hub/models--google--vit-base-patch16-224/snapshots/3f49326eb077187dfe1c2a2bb15fbd74e6ab91e3 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
提取图像特征:  29%|██▉       | 640/2194 [00:35<01:33, 16.65it/s]/root/miniconda3/lib/python3.12/site-packages/PIL/Image.py:1000: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
提取 BERT 文本特征: 100%|██████████| 69/69 [00:02<00:00, 28.24it/s]

BERT 文本特征提取完成。


In [4]:
from sklearn.model_selection import ParameterGrid
from torch import nn, optim
from sklearn.metrics import mean_squared_error, r2_score
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import ReLU, Sigmoid, LeakyReLU
import numpy as np
from tqdm import tqdm

# 定义评估指标函数
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)
    return mae, rmse, r2

# 定义 CustomDataset 类
class CustomDataset(Dataset):
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        feature = self.features[idx]
        feature = feature.reshape(1, -1)  # [sequence_length, input_size]
        return {
            'features': torch.tensor(feature, dtype=torch.float32),
            'targets': torch.tensor(self.targets[idx], dtype=torch.float32),
        }

# 定义 RNNRegressor 类
class RNNRegressor(nn.Module):
    def __init__(self, input_size, output_dim, activation_function, dropout, rnn_type='LSTM'):
        super(RNNRegressor, self).__init__()
        self.rnn_type = rnn_type
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(input_size, 64, 1, batch_first=True, dropout=dropout)  # 固定64个隐藏单元，1层
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(input_size, 64, 1, batch_first=True, dropout=dropout)  # 固定64个隐藏单元，1层
        else:
            self.rnn = nn.RNN(input_size, 64, 1, batch_first=True, nonlinearity='tanh', dropout=dropout)  # 固定64个隐藏单元，1层
        self.activation = activation_function
        self.fc = nn.Linear(64, output_dim)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        last_out = rnn_out[:, -1, :]  # [batch_size, hidden_size]
        activated = self.activation(last_out)
        out = self.fc(activated)
        return out

# 训练和评估模型
def train_and_evaluate_model(learning_rate, epochs, activation_function, random_seed, dropout, all_features, targets):
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)

    # 数据分割
    X_train, X_test, y_train, y_test = train_test_split(all_features, targets, test_size=0.01, random_state=random_seed)

    # 创建Dataset和DataLoader
    train_dataset = CustomDataset(X_train, y_train)
    test_dataset = CustomDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

    input_size = all_features.shape[1]
    output_dim = 1  # 回归任务

    # 初始化模型
    model = RNNRegressor(
        input_size=input_size,
        output_dim=output_dim,
        activation_function=activation_function,
        dropout=dropout,
        rnn_type='GRU'  # 'LSTM', 'GRU' or 'RNN'
    ).to(device)

    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # 训练模型
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for batch in train_loader:
            features = batch['features'].to(device)  # [batch_size, seq_length, input_size]
            targets_batch = batch['targets'].to(device).unsqueeze(1)  # [batch_size, 1]
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item() * features.size(0)
        epoch_loss /= len(train_loader.dataset)
        if (epoch + 1) % 100 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")

    # 测试模型
    model.eval()
    all_true = []
    all_predicted = []
    with torch.no_grad():
        for batch in test_loader:
            features = batch['features'].to(device)
            targets_batch = batch['targets'].to(device).unsqueeze(1)
            outputs = model(features)
            all_true.extend(targets_batch.cpu().numpy())
            all_predicted.extend(outputs.cpu().numpy())

    # 转换回原始收入值
    all_true = 10 ** np.array(all_true).flatten()
    all_predicted = 10 ** np.array(all_predicted).flatten()

    # 计算评估指标
    mae, rmse, r2 = evaluate_model(all_true, all_predicted)
    return mae, rmse, r2

# 网格搜索超参数
SEEDS = [42, 23, 15, 34, 18, 32, 47, 27, 8, 52]
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'epochs': [100, 200, 500],
    'activation_function': [ReLU(), Sigmoid(), LeakyReLU()],
}

best_params = None
best_r2 = float('-inf')
best_metrics = {}
results = []  # 保存所有组合和种子实验结果

# 固定的 dropout
fixed_dropout = 0.3  # 根据需求调整

In [ ]:
for params in tqdm(ParameterGrid(param_grid), desc="网格搜索超参数"):
    learning_rate = params['learning_rate']
    epochs = params['epochs']
    activation_function = params['activation_function']

    seed_metrics = []  # 保存当前超参数组合下所有随机种子的结果
    print(f"正在训练模型: {params}")
    for seed in SEEDS:
        mae, rmse, r2 = train_and_evaluate_model(
            learning_rate=learning_rate,
            epochs=epochs,
            activation_function=activation_function,
            random_seed=seed,
            dropout=fixed_dropout,
            all_features=all_features,
            targets=targets
        )
        seed_metrics.append({'seed': seed, 'MAE': mae, 'RMSE': rmse, 'R2': r2})
        print(f"Seed {seed} -> MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    # 计算平均值作为该超参数组合的最终指标
    avg_mse = np.mean([m['MAE'] for m in seed_metrics])
    avg_rmse = np.mean([m['RMSE'] for m in seed_metrics])
    avg_r2 = np.mean([m['R2'] for m in seed_metrics])
    results.append({
        'params': params,
        'avg_metrics': {'MAE': avg_mse, 'RMSE': avg_rmse, 'R2': avg_r2},
        'seed_metrics': seed_metrics
    })
    print(f"当前参数平均结果 -> MAE: {avg_mse:.4f}, RMSE: {avg_rmse:.4f}, R2: {avg_r2:.4f}")

    if avg_r2 > best_r2:
        best_r2 = avg_r2
        best_params = params
        best_metrics = {'MAE': avg_mse, 'RMSE': avg_rmse, 'R2': avg_r2}

print(f"最优超参数: {best_params}")
print(f"最佳评估结果 -> MSE: {best_metrics['MAE']:.4f}, RMSE: {best_metrics['RMSE']:.4f}, R2: {best_metrics['R2']:.4f}")

# 保存网格搜索结果为 CSV
import pandas as pd
results_df = pd.DataFrame([
    {
        'learning_rate': r['params']['learning_rate'],
        'epochs': r['params']['epochs'],
        'activation_function': r['params']['activation_function'].__class__.__name__,
        'avg_MSE': r['avg_metrics']['MAE'],
        'avg_RMSE': r['avg_metrics']['RMSE'],
        'avg_R2': r['avg_metrics']['R2'],
    }
    for r in results
])
results_df.to_csv('GRU_search_results.csv', index=False)
print("网格搜索结果已保存为 GRU_search_results.csv")

网格搜索超参数:   0%|          | 0/27 [00:00<?, ?it/s]/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


正在训练模型: {'activation_function': ReLU(), 'epochs': 100, 'learning_rate': 0.001}
Epoch [1/100], Loss: 0.9499
Epoch [100/100], Loss: 0.0015
Seed 42 -> MAE: 499.9879, RMSE: 998.3977, R2: 0.2270


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/100], Loss: 0.9362
Epoch [100/100], Loss: 0.0015
Seed 23 -> MAE: 361.7508, RMSE: 577.9914, R2: 0.5576


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


...

Epoch [1/500], Loss: 0.8407
Epoch [100/500], Loss: 0.0022
Epoch [200/500], Loss: 0.0005
Epoch [300/500], Loss: 0.0003
Epoch [400/500], Loss: 0.0018
Epoch [500/500], Loss: 0.0019
Seed 18 -> MAE: 1452.8065, RMSE: 2448.5676, R2: -0.5505


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/500], Loss: 1.3969
Epoch [100/500], Loss: 0.0021
Epoch [200/500], Loss: 0.0005
Epoch [300/500], Loss: 0.0002
Epoch [400/500], Loss: 0.0005
Epoch [500/500], Loss: 0.0026
Seed 32 -> MAE: 634.4076, RMSE: 1121.9413, R2: 0.6031


/root/miniconda3/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch [1/500], Loss: 1.1251
Epoch [100/500], Loss: 0.0024
Epoch [200/500], Loss: 0.0017
Epoch [300/500], Loss: 0.0044
Epoch [400/500], Loss: 0.0001
